In [1]:
import requests
import json
import pandas as pd
import numpy as np
import glob
import pycountry
import re

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Download open sanctions data as .json

## Import open sanctions data .json

In [3]:
with open('open_sanctions.json') as f:
    sanctions_data = f.readlines()

In [4]:
json.loads(sanctions_data[0])

{'id': 'acf-00040861bc3f593000830d987d09967ef3503ef1',
 'target': True,
 'first_seen': '2022-05-05T15:26:25',
 'last_seen': '2022-10-05T06:32:14',
 'schema': 'Person',
 'properties': {'notes': ['Russian propagandist: host of news program "Segodnia" ("Today") on NTV'],
  'position': ['Propagandists', 'Federal media employees'],
  'gender': ['male'],
  'name': ['Kolyvanov Egor'],
  'alias': ['Колыванов Егор'],
  'birthDate': ['1980-11-15']},
 'referents': [],
 'datasets': ['ru_acf_bribetakers'],
 'caption': 'Kolyvanov Egor'}

In [5]:
data=[]
for ent_index in range(len(sanctions_data)):
    data.append(json.loads(sanctions_data[ent_index]))

In [6]:
len(data)

431828

In [7]:
data[0]

{'id': 'acf-00040861bc3f593000830d987d09967ef3503ef1',
 'target': True,
 'first_seen': '2022-05-05T15:26:25',
 'last_seen': '2022-10-05T06:32:14',
 'schema': 'Person',
 'properties': {'notes': ['Russian propagandist: host of news program "Segodnia" ("Today") on NTV'],
  'position': ['Propagandists', 'Federal media employees'],
  'gender': ['male'],
  'name': ['Kolyvanov Egor'],
  'alias': ['Колыванов Егор'],
  'birthDate': ['1980-11-15']},
 'referents': [],
 'datasets': ['ru_acf_bribetakers'],
 'caption': 'Kolyvanov Egor'}

In [8]:
data=pd.DataFrame(data)

In [9]:
data.head()

,id,target,first_seen,last_seen,schema,properties,referents,datasets,caption
0,acf-00040861bc3f593000830d987d09967ef3503ef1,True,2022-05-05T15:26:25,2022-10-05T06:32:14,Person,{'notes': ['Russian propagandist: host of news...,[],[ru_acf_bribetakers],Kolyvanov Egor
1,acf-0011c68a768924609dc5da5707ac7fa4c4d645a2,True,2022-07-09T18:14:08,2022-10-05T06:32:14,Person,"{'name': ['Shipov Sergei Yurievich'], 'notes':...",[],[ru_acf_bribetakers],Shipov Sergei Yurievich
2,acf-001e7e4c0363f08f1e784c230457960b84a6416f,True,2022-05-05T15:26:25,2022-10-05T06:32:14,Person,"{'name': ['Egorov Ivan Mikhailovich'], 'alias'...",[],[ru_acf_bribetakers],Egorov Ivan Mikhailovich
3,acf-002c208139012c8d93b6298358188d7cadafe648,True,2022-09-20T01:05:26,2022-10-05T06:32:14,Person,"{'name': ['Goreslavsky Alexey Sergeyevich'], '...",[],[ru_acf_bribetakers],Goreslavsky Alexey Sergeyevich
4,acf-002cc8fdf8fe41185091a7cb6c598663e7a22eb5,True,2022-09-20T01:05:26,2022-10-05T06:32:14,Person,"{'gender': ['female'], 'name': ['Samoilova Nat...",[],[ru_acf_bribetakers],Samoilova Natalya Vladimirovna


In [10]:
data.drop(['target','first_seen','last_seen','datasets'],1,inplace=True)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [11]:
unique_keys=[]
for row in data['properties']:
    for key in row.keys():
        if key not in unique_keys:
            unique_keys.append(key)

In [12]:
# Only keep entities with an entry on the name field
df_dict={}
for df_dict_index,row in enumerate(data['properties']):
    for key in unique_keys:
        if key not in row:
            row[key]=''    
    if row['name']:
        df_dict[df_dict_index]=row 

In [13]:
len(df_dict)

244612

In [14]:
data.drop(['properties','referents'],1,inplace=True)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [15]:
properties=pd.DataFrame.from_dict(df_dict,orient='index')

In [16]:
properties.shape

(244612, 112)

In [17]:
properties=properties.replace('',np.nan)

In [18]:
# Columns made up entirely of NaN
cols_to_drop=properties.isna().all(0)

In [19]:
cols_to_drop=list(cols_to_drop[cols_to_drop.values].index)

In [20]:
properties.drop(cols_to_drop,1,inplace=True)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [21]:
# Create copy of wikidataIDs
wikidataIDs=properties['wikidataId'].copy()
wikidataIDs[~wikidataIDs.isna()]=wikidataIDs[~wikidataIDs.isna()].apply(lambda x: ''.join(x))

In [22]:
# Create copy of websites
websites=properties['website'].copy()
websites[~websites.isna()]=websites[~websites.isna()].apply(lambda x: ''.join(x))

In [23]:
# manually selected columns to drop
more_cols_to_drop=[
                'bikCode',
                'dunsCode',
                'callSign',
                'tonnage',
                'grossRegisteredTonnage', 
                'ogrnCode', 
                'innCode',
                'leiCode',
                'swiftBic',
                'ogrnCode', 
                'classification', 
                'program', 
                'sourceUrl', 
                'addressEntity', 
                'imoNumber', 
                'mmsi',
                'registrationNumber',
                'modifiedAt',
                'idNumber',
                'passportNumber',
                'phone',
                'kppCode',
                'vatCode',
                'serialNumber',
                'owner',
                'opencorporatesUrl',
                'taxNumber',
                'flag',
                'status',
                'jurisdiction',
                'wikidataId',
                'email',
                'website',
                'education',
                'type',
                'firstName',
                'secondName',
                'createdAt',
                'middleName',
                'lastName',
                'title',
                'religion',
                'buildDate',
                'model',
                'incorporationDate',
                'previousName',
                'fatherName',
                'motherName',
                'address',
                'legalForm',
                ]

In [24]:
properties.drop(more_cols_to_drop,1,inplace=True)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [25]:
# Convert topics into sentences
topics_translation_dict={
    'crime':'Crime',
    'crime.fraud':'Fraud',
    'crime.cyber':'Cybercrime',
    'crime.fin':'Financial crime',
    'crime.theft':'Theft',
    'crime.war':'War crimes',
    'crime.boss':'Criminal leadership',
    'crime.terror':'Terrorism',
    'crime.traffick':'Trafficking',
    'crime.traffick.drug':'Drug trafficking',
    'crime.traffick.human':'Human trafficking',
    'corp.offshore':'Offshore',
    'corp.shell':'Shell company',
    'gov':'Government',
    'gov.national':'National government',
    'gov.state':'State government',
    'gov.muni':'Municipal government',
    'gov.soe':'State-owned enterprise',
    'gov.igo':'Intergovernmental organization',
    'fin':'Financial services',
    'fin.bank':'Bank',
    'fin.fund':'Fund',
    'fin.adivsor':'Financial advisor',
    'role.pep':'Politician',
    'role.rca':'Close Associate',
    'role.judge':'Judge',
    'role.civil':'Civil servant',
    'role.diplo':'Diplomat',
    'role.lawyer':'Lawyer',
    'role.acct':'Accountant',
    'role.spy':'Spy',
    'role.oligarch':'Oligarch',
    'role.journo':'Journalist',
    'role.act':'Activist',
    'pol.party':'Political party',
    'pol.union':'Union',
    'rel':'Religion',
    'mil':'Military',
    'asset.frozen':'Frozen asset',
    'sanction':'Sanctioned entity',
    'debarment':'Debarred entity',
    'poi':'Person of interest'}

for key,value in topics_translation_dict.items():
    value=f'Associated with {value}.'
    topics_translation_dict[key]=value

properties['topics']=properties['topics'].fillna('').apply(lambda x: [topics_translation_dict[key] for key in x])
properties['topics']=properties['topics'].apply(lambda x: ' '.join(x))

In [26]:
def transform_into_sentence(df,col,sentence,separator=', '): 
    df.loc[~df[col].isna(),col]=df.loc[~df[col].isna(),col].apply(lambda x: f'{separator}'.join(x))
    df.loc[~df[col].isna(),col]=df.loc[~df[col].isna(),col].apply(lambda x: f'{sentence}{x}.')

In [27]:
def show_not_na(df,col,n_rows=20):
    return df[~df[col].isna()].head(n_rows)

In [28]:
data.columns

Index(['id', 'schema', 'caption'], dtype='object')

In [29]:
# Remove dates and ordinals from each string in the list
date_expr = re.compile('\d{4}-\d{4}')
digit_expr = re.compile('\s\d{2}[a-zA-Z]{2}\s')
for expr in [date_expr, digit_expr]:
    # Delete expression from string
    properties.loc[~properties['position'].isna(),'position']=properties.loc[~properties['position'].isna(),'position'].apply(lambda x: [re.sub(expr, ' ', i)  for i in x])
    # Remove parentheses and comma
    properties.loc[~properties['position'].isna(),'position']=properties.loc[~properties['position'].isna(),'position'].apply(lambda x: [i.replace('(','').replace(')','').replace(',','') for i in x])
    # Remove position redundancy
    properties.loc[~properties['position'].isna(),'position']=properties.loc[~properties['position'].isna(),'position'].apply(lambda x: set(x))

In [30]:
# export dates
properties['birthdate']=properties['birthDate']
properties['deathdate']=properties['deathDate']

In [31]:
cols_to_sentence={
    'gender':'This person is a ',
    'position':'This person has held these positions: ',
    'birthDate':'This person was born in ',
    'birthPlace': 'This person was born in ',
    'deathDate':'This person died in ',
    'keywords':'This person has worked in: ',
    'sector':'This person worked for: ',
    'publisher':'This person was present in ',
    'pastFlags':'In the past this person was at ',
    'ethnicity':'This person\'s ethnicity is '
}
for col,sentence in cols_to_sentence.items():
    transform_into_sentence(properties,col,sentence)

In [32]:
# Fix most common position abbreviations 
positions_in_full={'Min\.':'Minister','Dep\.':'Deputy','Pres\.':'President', 'Dir\.':'Director'}
for abbv, full in positions_in_full.items():
    properties['position']=properties['position'].str.replace(abbv, full)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


In [33]:
# Convert country ISO alpha 2 codes into names

In [34]:
for regionality in ['country','nationality']:
    properties.loc[~properties[regionality].isna(),regionality]=\
        properties.loc[~properties[regionality].isna(),regionality].apply(lambda x: [country for country in x if len(country)==2])

    exceptions=set()
    countries=set()
    for row in properties.loc[~properties[regionality].isna(),regionality]:
        for country in row:
            try:
                countries.add(pycountry.countries.get(alpha_2=country).name)
            except:
                exceptions.add(country)

    properties.loc[~properties[regionality].isna(),regionality]=\
        properties.loc[~properties[regionality].isna(),regionality].apply(lambda x: 
                                ','.join([pycountry.countries.get(alpha_2=country).name
                                 for country in x if country not in exceptions])
                               )

In [35]:
# Transform country and nationality into sentences
properties.loc[~properties['country'].isna(),'country']=properties.loc[~properties['country'].isna(),'country'].apply(lambda x: f'This person belongs to these countries: {x}.')
properties.loc[~properties['nationality'].isna(),'nationality']=properties.loc[~properties['nationality'].isna(),'nationality'].apply(lambda x: f'This person has these nationalities: {x}.')

In [36]:
properties['AKA'] = properties['name'] + properties['alias'] + properties['weakAlias']

In [37]:
properties.drop(['name','alias', 'weakAlias'],1,inplace=True)

/Users/luis_flores/.local/share/virtualenvs/ner-DhZLIlym/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [38]:
context_cols=[
    'position', 
    'gender',
    'birthDate', 
    'country',
    'topics', 
    'birthPlace', 
    'nationality', 
    'sector', 
    'keywords', 
    'deathDate',
    'publisher',
    'pastFlags',  
    'ethnicity'
]

In [39]:
i = 0
for col in context_cols:
    if i==0:
        properties['context']=properties[col].fillna('NAN')
        i+=1
    else:
        properties['context']= properties['context'] + properties[col].fillna('NAN')
properties['context']=properties['context'].str.replace('NAN','')
properties['context']=properties['context'].str.split('.').apply(lambda x:'. '.join(x))

In [40]:
properties=properties[['notes','AKA','context','birthdate','deathdate']]

In [41]:
properties['notes']=properties['notes'].fillna('').apply(lambda x:' '.join(x))

In [42]:
properties=properties.merge(wikidataIDs,left_index=True, right_index=True)
properties=properties.merge(websites,left_index=True, right_index=True)

In [43]:
data=data.merge(properties,left_index=True,right_index=True)

In [44]:
# Remove dates from list
data.loc[~data['birthdate'].isna(),'birthdate']=data.loc[~data['birthdate'].isna(),'birthdate'].apply(lambda x: x[0])
data.loc[~data['deathdate'].isna(),'deathdate']=data.loc[~data['deathdate'].isna(),'deathdate'].apply(lambda x: x[0])

In [45]:
data['schema'].value_counts()

Person          218690
Company          16560
LegalEntity       4893
Organization      3659
Vessel             524
Airplane           286
Name: schema, dtype: int64

In [46]:
data=data[data['schema']=='Person']

In [47]:
data=data.rename(columns={'caption':'name'})

In [48]:
data.shape

(218690, 10)

In [49]:
# Include name in context info
empty_notes_indices=data[data['notes'].str.replace(' ','').str.len()==0].index.values
filled_notes_indices=data[data['notes'].str.replace(' ','').str.len()>0].index.values
# Include name in description when context is null
data.loc[empty_notes_indices, 'notes'] = data.loc[empty_notes_indices,'name'].apply(lambda x: f'This person is called {x}.')
# Include name in description when context is not null
data.loc[filled_notes_indices,'notes']=data.loc[filled_notes_indices, 'name'].apply(lambda x: f'{x} was linked to ') + data.loc[filled_notes_indices,'notes'] + '.'

In [50]:
data['full_notes']= data['notes'].fillna('') + ' ' + data['context'].fillna('')

In [51]:
data.to_csv('../../kb_datasets/open_sanctions_entities.csv')

In [ ]:
#######

In [105]:
data['first_name']=data['name'].apply(lambda r: r.split(' ')[0])

In [106]:
data['last_name']=data['name'].apply(lambda r: r.split(' ')[-1])

In [107]:
first_name_counts=data.groupby(['first_name'])['first_name'].count()

In [108]:
first_name_counts=first_name_counts.to_frame(name='counts').reset_index()

In [109]:
first_name_counts.sort_values(by='counts', ascending=False).head(10)

,first_name,counts
63039,ОБЩЕСТВО,5179
24804,John,1201
24970,José,1122
12817,David,1052
35518,Michael,774
64241,ФЕДЕРАЛЬНОЕ,764
42517,Peter,762
34637,María,755
34240,Maria,633
45493,Robert,579


In [110]:
last_name_counts=data.groupby(['last_name'])['last_name'].count()

In [111]:
last_name_counts=last_name_counts.to_frame(name='counts').reset_index()

In [112]:
last_name_counts.sort_values(by='counts', ascending=False).head(10)

,last_name,counts
57077,LIMITED,931
14318,Bank,875
11078,Association,551
57226,LTD,535
103295,Vladimirovich,515
73375,Nikolaevich,390
8608,Alexandrovich,334
114152,"ДОРОГИ""",321
25265,Company,316
52773,Khan,303


In [113]:
same_name_counts=data.groupby(['first_name','last_name'])['first_name'].count()

In [114]:
same_name_counts=same_name_counts.to_frame(name='counts').reset_index()

In [115]:
same_name_counts.sort_values(by='counts', ascending=False).head(100)

,first_name,last_name,counts
216501,ЧАСТНОЕ,"ДОРОГИ""",258
216120,ФЕДЕРАЛЬНОЕ,ФЕДЕРАЦИИ,150
211286,ОБЩЕСТВО,"ТЕХНОЛОГИИ""",102
71400,Hon.,MP,101
216121,ФЕДЕРАЛЬНОЕ,"ФЕДЕРАЦИИ""",101
55720,First,Bank,92
216091,ФЕДЕРАЛЬНОЕ,"ОБЛАСТИ""",72
176187,The,Bank,68
210634,ОБЩЕСТВО,"КОМПАНИЯ""",67
206688,НЕГОСУДАРСТВЕННОЕ,"ДОРОГИ""",62


In [116]:
data[(data['first_name']=='John')&(data['last_name']=='Williams')]

,id,schema,name,notes,AKA,context,full_notes,first_name,last_name
160001,Q104815415,Person,John Williams,American politician,"[John Williams, জন উইলিয়ামস]",Member of the West Virginia House of Delegates...,American politician Member of the West Virgini...,John,Williams
207004,Q1701350,Person,John Williams,American politician,"[John Williams, John Patrick Williams, Пэт Уил...",United States representative (1983-1985) Unite...,American politician United States representati...,John,Williams
234990,Q3182766,Person,John G. Williams,Canadian politician,"[John G. Williams, جون جى. وليامز, John Willia...",member of the House of Commons of Canada male ...,Canadian politician member of the House of Com...,John,Williams
272665,Q6264317,Person,John Williams,New South Wales federal politician,"[John Williams, Реджинальд Уильямс, 約翰·威廉斯, Se...","Senate (member, 2007-2010) Senate (member, 201...",New South Wales federal politician Senate (mem...,John,Williams
272666,Q6264319,Person,John Williams,Australian politician,"[John Williams, Джон Уильямс, জন উইলিয়ামস, جو...",Member of the New South Wales Legislative Asse...,Australian politician Member of the New South ...,John,Williams
412498,us-occ-fab7e3e76aa4cbe7cda3a1223d5cc1efa91a5199,Person,John Williams,,NaN,us crime.fin John Williams,us crime.fin John Williams,John,Williams


In [51]:
os_ents=set(data['name'])
unique_gu_ents=set()
for gu_ner_file in glob.glob('data/extracted_named_entities_output/sampled_*'):
    gu_ents = pd.read_csv(gu_ner_file,index_col=0)
    unique_gu_ents = unique_gu_ents | set(gu_ents['text'])

In [52]:
common_ents=[ent for ent in unique_gu_ents if ent in os_ents]

In [53]:
len(common_ents)

12079

In [54]:
common_gu_ents=''
first_cycle=1
for gu_ner_file in glob.glob('data/extracted_named_entities_output/sampled_*'):
    year=gu_ner_file.split('_')[-2]
    gu_ents = pd.read_csv(gu_ner_file,index_col=0)
    gu_ents['article_year']=year
    gu_ents=gu_ents[gu_ents['text'].isin(common_ents)]
    if first_cycle:
        common_gu_ents=gu_ents
        first_cycle=0
    else:
        common_gu_ents=pd.concat([common_gu_ents, gu_ents],ignore_index=True)

In [55]:
common_gu_ents

,text,label,start,end,start_char,end_char,doc_index,article_year
0,Sonia Sotomayor,PERSON,7,9,42,57,4,2022
1,Samuel Alito,PERSON,151,153,830,842,4,2022
2,Donald Trump,PERSON,193,195,1067,1079,4,2022
3,John Roberts,PERSON,514,516,2655,2667,4,2022
4,Wells Fargo,ORG,39,41,227,238,8,2022
...,...,...,...,...,...,...,...,...
320540,Moon Jae-in,PERSON,246,250,1263,1274,17131,2021
320541,Yong Hye-in,PERSON,430,434,2206,2217,17131,2021
320542,António Guterres,PERSON,69,71,367,383,17133,2021
320543,Antony Blinken,PERSON,399,401,2174,2188,17133,2021


In [56]:
common_ents_data=common_gu_ents.merge(data, left_on='text', right_on='name')

In [57]:
common_ents_data.drop(['text'],1,inplace=True)

/tmp/ipykernel_1426/3683316261.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  common_ents_data.drop(['text'],1,inplace=True)


In [58]:
common_ents_data.loc[common_ents_data['label']=='ORG','name'].nunique()

236

In [59]:
common_ents_data.loc[common_ents_data['label']=='PERSON','name'].nunique()

11917

In [60]:
common_ents_data.loc[common_ents_data['label']=='ORG']

,label,start,end,start_char,end_char,doc_index,article_year,id,schema,name,context
11932,ORG,273,275,1397,1409,10201,2017,Q22686,Person,Donald Trump,President of the United States from 2017 to 20...
14529,ORG,39,41,227,238,8,2022,us-occ-1741-wells-fargo,Company,Wells Fargo,us fin.bank
14530,ORG,39,41,227,238,8,2022,us-occ-17441-wells-fargo,Company,Wells Fargo,us fin.bank
14531,ORG,329,331,1777,1788,8,2022,us-occ-1741-wells-fargo,Company,Wells Fargo,us fin.bank
14532,ORG,329,331,1777,1788,8,2022,us-occ-17441-wells-fargo,Company,Wells Fargo,us fin.bank
...,...,...,...,...,...,...,...,...,...,...,...
339838,ORG,66,69,354,383,8743,2021,us-occ-518072-regions-financial-corporation,Company,Regions Financial Corporation,us fin.bank
339839,ORG,4889,4892,26087,26116,8996,2021,us-occ-518072-regions-financial-corporation,Company,Regions Financial Corporation,us fin.bank
340059,ORG,4062,4063,21241,21245,16105,2021,NK-4KEadPotHHwwz9pfX3DaLx,Vessel,NESA,Subject to Secondary Sanctions (Linked To: NAT...
340060,ORG,4195,4196,21972,21976,16105,2021,NK-4KEadPotHHwwz9pfX3DaLx,Vessel,NESA,Subject to Secondary Sanctions (Linked To: NAT...


In [61]:
gu_body_texts=''
first_cycle=1
for gu_file in glob.glob('data/sampled_*'):
    year=gu_file.split('_')[-1].split('.')[-2]
    gu_article=pd.read_csv(gu_file)
    gu_article=gu_article[['body_text']]
    gu_article['article_year']=int(year)
    gu_article['doc_index']=gu_article.index.values
    if first_cycle:
        gu_body_texts=gu_article
        first_cycle=0
    else:
        gu_body_texts=pd.concat([gu_body_texts, gu_article],ignore_index=True)

In [62]:
common_ents_data['article_year']=common_ents_data['article_year'].astype(int)

In [63]:
common_ents_data=common_ents_data.merge(gu_body_texts, how='left', on=['article_year','doc_index'])

In [64]:
common_ents_data[common_ents_data['body_text'].isna()]

,label,start,end,start_char,end_char,doc_index,article_year,id,schema,name,context,body_text


In [66]:
common_ents_data.to_csv('opensanctions_gu_intersection_common_ents_data.csv')

In [103]:
common_ents_data.head()

,label,start,end,start_char,end_char,doc_index,article_year,id,schema,name,context,body_text
0,PERSON,7,9,42,57,4,2022,Q11107,Person,Sonia Sotomayor,US Supreme Court justice since 2009 Judge of t...,The liberal-leaning supreme court justice Soni...
1,PERSON,524,526,2780,2795,28,2022,Q11107,Person,Sonia Sotomayor,US Supreme Court justice since 2009 Judge of t...,"In a barbed intervention on Saturday, the Whit..."
2,PERSON,366,368,1881,1896,4778,2022,Q11107,Person,Sonia Sotomayor,US Supreme Court justice since 2009 Judge of t...,The extremist Colorado Republican congresswoma...
3,PERSON,443,445,2384,2399,4953,2022,Q11107,Person,Sonia Sotomayor,US Supreme Court justice since 2009 Judge of t...,The US supreme court has sided with Republican...
4,PERSON,62,64,362,377,5179,2022,Q11107,Person,Sonia Sotomayor,US Supreme Court justice since 2009 Judge of t...,The supreme court will not allow the Biden adm...


In [105]:
common_ents_data[['body_text','name','label','article_year','doc_index','id','context']].sample(20)

,body_text,name,label,article_year,doc_index,id,context
37560,"“Texas.” It was said with incredulity, by one ...",Ted Cruz,PERSON,2022,8830,Q2036942,"American politician, United States senator fro..."
315891,We end the night with nothing having changed i...,Susan Lamb,PERSON,2018,17718,Q25324887,Australian politician (born 1972) Member of th...
43141,"Standing on the lawn of the US Capitol, in cle...",Joe Biden,PERSON,2021,8781,Q6279,President of the United States since 2021 Sena...
297074,"They’ve tried dams, dykes and dialogue over th...",Shavkat Mirziyoyev,PERSON,2018,5684,Q57808,President of Uzbekistan (2016-present) Prime M...
122939,It was the equivalent of a V-sign cheerfully f...,Sadiq Khan,PERSON,2019,12185,Q334155,"British politician, Mayor of London (born 1970..."
11364,We’re going to wrap up our live coverage of th...,Donald Trump,PERSON,2017,6719,Q22686,President of the United States from 2017 to 20...
153550,Read David Hytner’s report from the Emirates S...,Arsenal,ORG,2018,11874,trade-csl-175dd69cf132de58796c2fc46dc4420c87cc...,ru
186301,Murderous suicide bombings. A deadly upsurge o...,Bashar al-Assad,PERSON,2016,22766,Q44329,President of the Republic; person authorising ...
211750,"After quite a day, I will leave it there. Here...",Christian Porter,PERSON,2021,16754,Q5110039,Australian politician Member of the Western Au...
265350,The House Tom Watson and Imogen Robertson Sphe...,Ann Widdecombe,PERSON,2020,1319,Q298846,British politician (born 1947) House of Common...


In [109]:
common_ents_data.loc[(common_ents_data['doc_index']==8830)&(common_ents_data['article_year']==2018),['body_text','name','label','article_year','doc_index','id','context']]

,body_text,name,label,article_year,doc_index,id,context
1269,"Good morning, I’m Tim Walker with today’s head...",Donald Trump,PERSON,2018,8830,Q22686,President of the United States from 2017 to 20...
1270,"Good morning, I’m Tim Walker with today’s head...",Donald Trump,PERSON,2018,8830,Q22686,President of the United States from 2017 to 20...
15226,"Good morning, I’m Tim Walker with today’s head...",David Smith,PERSON,2018,8830,Q3018800,Quebec politician member of the House of Commo...
15227,"Good morning, I’m Tim Walker with today’s head...",David Smith,PERSON,2018,8830,Q5239878,Canadian senator member of the House of Common...
15228,"Good morning, I’m Tim Walker with today’s head...",David Smith,PERSON,2018,8830,Q53960880,Australian Capital Territory politician Senate...
27819,"Good morning, I’m Tim Walker with today’s head...",Brett Kavanaugh,PERSON,2018,8830,Q4962244,Associate Justice of the Supreme Court of the ...
115813,"Good morning, I’m Tim Walker with today’s head...",Jacinda Ardern,PERSON,2018,8830,Q3606816,Prime Minister of New Zealand since 2017 Membe...
119932,"Good morning, I’m Tim Walker with today’s head...",Rodrigo Duterte,PERSON,2018,8830,Q457786,Filipino politician and the 16th President of ...
303900,"Good morning, I’m Tim Walker with today’s head...",Antonio Trillanes IV,PERSON,2018,8830,Q525412,Philippine politician and navy officer Member ...


In [112]:
common_ents_data[common_ents_data['article_year']==2022].to_csv('opensanctions_gu_intersection_common_ents_data_2022.csv')

In [67]:
from difflib import SequenceMatcher

In [68]:
from itertools import permutations

In [87]:
data['name'].nunique()

239865

In [88]:
data['name'].nunique()

239865

In [96]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
data

In [45]:
data['properties'][0]

{'name': ['Kolyvanov Egor'],
 'alias': ['Колыванов Егор'],
 'notes': ['Russian propagandist: host of news program "Segodnia" ("Today") on NTV'],
 'position': ['Federal media employees', 'Propagandists'],
 'gender': ['male'],
 'birthDate': ['1980-11-15'],
 'program': ''}

In [39]:
pd.DataFrame(columns=unique_keys)

,name,alias,notes,position,gender,birthDate,program,authority,sourceUrl,reason,...,wikidataId,education,religion,ethnicity,sharesCount,associate,amountUsd,balance,sharesValue,date


## Dataset stats

In [57]:
# unique schemas in dataset:
unique_schemas=[]
for ent in data:
    if ent['schema'] not in unique_schemas:
        unique_schemas.append(ent['schema'])

In [58]:
unique_schemas

['Person',
 'Sanction',
 'LegalEntity',
 'Address',
 'Passport',
 'Identification',
 'Organization',
 'UnknownLink',
 'Family',
 'Directorship',
 'Company',
 'Membership',
 'Vessel',
 'Ownership',
 'Representation',
 'Security',
 'CryptoWallet',
 'Airplane',
 'Associate']

In [71]:
data_person=[]
for ent_index in range(len(data)):
    if data[ent_index]['schema']=='Person':
        data_person.append(data[ent_index])

In [73]:
len(data_person)

218972

In [75]:
data_person[random.randint(0, len(data_person))]

{'id': 'evpo-458196f2-cd2d-4a29-8e61-8e811982ad44',
 'target': True,
 'first_seen': '2021-07-26T11:55:45',
 'last_seen': '2022-04-25T12:21:45',
 'schema': 'Person',
 'properties': {'nationality': ['ph'],
  'name': ['Albano, Rodolfo III T.'],
  'gender': ['male'],
  'topics': ['role.pep'],
  'position': ['House of Representatives (member, 2016-)',
   'House of Representatives (member, 2013-2016)'],
  'modifiedAt': ['2019-05-09']},
 'referents': [],
 'datasets': ['everypolitician'],
 'caption': 'Albano, Rodolfo III T.'}

In [76]:
data_person[random.randint(0, len(data_person))]

{'id': 'Q101002571',
 'target': True,
 'first_seen': '2022-01-17T01:29:38',
 'last_seen': '2022-09-09T06:44:49',
 'schema': 'Person',
 'properties': {'name': ['Tadeja Šuštar'],
  'topics': ['role.pep'],
  'country': ['si'],
  'wikidataId': ['Q101002571'],
  'notes': ['Slovene politician'],
  'gender': ['female'],
  'birthPlace': ['Ljubljana'],
  'nationality': ['si'],
  'firstName': ['Tadeja'],
  'lastName': ['Šuštar'],
  'position': ['Member of the National Assembly of Slovenia (2019-)'],
  'modifiedAt': ['2022-04-15'],
  'birthDate': ['1990-09-14']},
 'referents': [],
 'datasets': ['wikidata', 'wd_peppercat_legislators'],
 'caption': 'Tadeja Šuštar'}

In [77]:
data_person[random.randint(0, len(data_person))]

{'id': 'Q1633248',
 'target': True,
 'first_seen': '2022-03-06T02:00:34',
 'last_seen': '2022-09-09T06:44:49',
 'schema': 'Person',
 'properties': {'wikidataId': ['Q1633248'],
  'name': ['Hubert Kinz'],
  'notes': ['Austrian politician and lawyer'],
  'gender': ['male'],
  'nationality': ['at'],
  'firstName': ['Hubert'],
  'birthPlace': ['Bregenz'],
  'topics': ['role.pep'],
  'position': ['Member of Landtag of Vorarlberg (2009-)'],
  'education': ['University of Innsbruck (1979-1983)'],
  'alias': ['Hubert Kinz'],
  'lastName': ['Kinz'],
  'country': ['at'],
  'modifiedAt': ['2022-04-15'],
  'birthDate': ['1960-10-06']},
 'referents': [],
 'datasets': ['wikidata', 'wd_peppercat_legislators'],
 'caption': 'Hubert Kinz'}

In [91]:
if [2]:
    print('okay')

okay


In [92]:
data_person_has_notes=[]
for person in data_person:
    try:
        if person['properties']['notes']:
            data_person_has_notes.append(person)
    except:
        pass

In [93]:
len(data_person_has_notes)

160599

In [ ]:
pd.DataFrame(ent['properties'])